# 3 Prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn

%load_ext autoreload
%autoreload 2

from a02_helper import *
from a02_functions import gd, predict, classify, optimize

In [ ]:
# Fitted model
w0 = np.random.normal(size=D)
wz_gd, vz_gd, ez_gd = optimize(gd(y, Xz), w0, nepochs=500)

In `a02_functions.py`, complete the `predict` and `classify` methods for the predicted
spam probability and predicted class label, respectively. Use them to explore your
previously fitted model.

In [ ]:
# Exploration example: confusion matrix
yhat = predict(Xtestz, wz_gd)
ypred = classify(Xtestz, wz_gd)
print(sklearn.metrics.confusion_matrix(ytest, ypred))  # true x predicted

In [ ]:
# Exploration example: classification report
print(sklearn.metrics.classification_report(ytest, ypred))

In [ ]:
# Exploration Example: precision-recall curve (with annotated thresholds)
nextplot()
precision, recall, thresholds = sklearn.metrics.precision_recall_curve(ytest, yhat)
plt.plot(recall, precision)
for x in np.linspace(0, 1, 10, endpoint=False):
    index = int(x * (precision.size - 1))
    plt.text(recall[index], precision[index], "{:3.2f}".format(thresholds[index]))
plt.xlabel("Recall")
plt.ylabel("Precision")

In [ ]:
# Explore which features are considered important
# YOUR CODE HERE